<a href="https://colab.research.google.com/github/ep657/Exercise-1-A-general-Huckel-solver/blob/main/Huckel_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import sys
# By setting α to 0 and β to 1, the Huckel Hamiltonian can be reduced to the adjacency matrix
# For the linear and cyclic cases, it was straightforward to write a program which generates the adjacency matrix for a molecule containing a given number of atoms
# For the sp2-hybridized Platonic solids and Buckminsterfullerene, I was not able to write a program which would generate their adjacency matrices, so they are simply hard-coded
# hence this dictionary of preset adjacency matrices
# The adjacency matrices for the sp2 hybridised Platonic solids are taken from:
# www.distanceregular.org/indexes/upto50vertices.html
# The adjacency matrix for Buckminsterfullerene is taken from:
# math.stackexchange.com/questions/4477058/adjacency-matrix-for-soccer-ball-football

PRESETS = {
    'tetrahedron': [
        [0,1,1,1],
        [1,0,1,1],
        [1,1,0,1],
        [1,1,1,0]
    ],
    'octahedron': [
        [0,1,1,1,1,0],
        [1,0,1,1,0,1],
        [1,1,0,0,1,1],
        [1,1,0,0,1,1],
        [1,0,1,1,0,1],
        [0,1,1,1,1,0]
    ],
    'cube': [
        [0,0,0,0,0,1,1,1],
        [0,0,0,0,1,0,1,1],
        [0,0,0,0,1,1,0,1],
        [0,0,0,0,1,1,1,0],
        [0,1,1,1,0,0,0,0],
        [1,0,1,1,0,0,0,0],
        [1,1,0,1,0,0,0,0],
        [1,1,1,0,0,0,0,0]
    ],
     'icosahedron': [
    [0,1,0,0,1,0,1,1,0,0,1,0],
    [1,0,0,0,1,1,0,1,0,1,0,0],
    [0,0,0,1,0,0,1,0,1,0,1,1],
    [0,0,1,0,0,0,1,1,0,1,0,1],
    [1,1,0,0,0,1,0,0,1,0,1,0],
    [0,1,0,0,1,0,0,0,1,1,0,1],
    [1,0,1,1,0,0,0,1,0,0,1,0],
    [1,1,0,1,0,0,1,0,0,1,0,0],
    [0,0,1,0,1,1,0,0,0,0,1,1],
    [0,1,0,1,0,1,0,1,0,0,0,1],
    [1,0,1,0,1,0,1,0,1,0,0,0],
    [0,0,1,1,0,1,0,0,1,1,0,0]
],
      'dodecahedron': [
    [0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0],
    [1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0],
    [1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0],
    [1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0],
    [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0],
    [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1],
    [0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
    [0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1],
    [0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0],
    [0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1],
    [0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0],
    [0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0]
],
     'fullerene60': [
    [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1],
    [0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
    [1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
    [0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1],
    [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1],
    [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0]
]}

In [ ]:
def welcome_message():
    print("----------------")
    print("Program to determine the Huckel π-energies and degeneracies for a range of molecules")

def commands():
    print("----------------")
    print("Available commands:\n")
    print("linear\tto determine the Huckel π-energies and degeneracies for a linear polyene with n carbons")
    print("cyclic\tto determine the Huckel π-energies and degeneracies for a cyclic polyene with n carbons")
    print("presets\tto determine the Huckel π-energies and degeneracies for a set of preset molecules including certain sp2-hybridized Platonic solids and Buckminsterfullerene")
    print("quit\tto exit the program")

# This function calculates the eigenvalues of the adjacency matrix
# The eigenvalues are then sorted in ascending order
# This ordered list of eigenvalues is then multiplied by -1 to get the actual ordering of the energy levels
def calculate_eigenvalues(adj_matrix):
    """Calculate eigenvalues of the adjacency matrix"""
    eigenvalues, _ = np.linalg.eig(adj_matrix)
    eigenvalues.sort()
    return -1 * eigenvalues

# This function iterates through the sorted eigenvalues, checking if eigenvalue i is within epsilon of eigenvalue i+1
# in which case eigenvalues i and i+1 are considered degenerate
# The  function returns a list of tuples, where each tuple contains a unique eigenvalue and its degeneracy count
def calculate_degeneracies(eigenvalues):
    """Calculate degeneracies for each eigenvalue"""
    epsilon = 10e-7
    degeneracy = 1
    result = []
    for i in range(eigenvalues.size - 1):
        if (eigenvalues[i + 1] - epsilon) < eigenvalues[i] < (eigenvalues[i + 1] + epsilon):
            degeneracy += 1
        else:
            result.append((eigenvalues[i], degeneracy))
            degeneracy = 1
    result.append((eigenvalues[-1], degeneracy))
    return result

# This function returns the adjacency matrix for a linear polyene consisting of n atoms
# The adjacency matrix for a linear polyene should have ones along the super- and sub-diagonals and zeros everywhere else
# Hence the adjacency matrix for a linear polyene can be defined as the sum of a matrix with ones only on the superdiagonal and a matrix with ones only on the subdiagonal
def adjacency_linear(n):
    """Create adjacency matrix for a linear polyene consisting of n atoms"""
    return np.eye(n, n, 1) + np.eye(n, n, -1)

# The adjacency matrix for a cyclic polyene is almost the same as that for a linear polyene
# but since the first and last atoms are now connected, we must add in ones to the top left and bottom right corners
def adjacency_cyclic(n):
    """Create adjacency matrix for cyclic polyene consisting of n atoms"""
    return adjacency_linear(n) + np.eye(n, n, n - 1) + np.eye(n, n, -n + 1)

# Printing the calculated energy levels with their associated degeneracies
# The eigenvalues are printed to 3dp
def print_results(adjacency_matrix):
    """Print the calculated Huckel energies and degeneracies"""
    eigenvals = calculate_eigenvalues(adjacency_matrix)
    degeneracy_eigenvals = calculate_degeneracies(eigenvals)
    print(f'{"Energy":10}', f'{"Degeneracy"}')
    for energy, degeneracy in degeneracy_eigenvals:
        print(f"α{-energy.real:+.3f}*β", f'{degeneracy:^10}')

# This function allows the user to input their choice of molecule
# When the user inputs their command, the appropriate function will be called
def connectivity_input(command, n):
    if command == "linear":
        return adjacency_linear(n)
    elif command == "cyclic":
        return adjacency_cyclic(n)
    elif command == "presets":
        show_presets()
        select_preset()
    elif command in PRESETS.keys():
        return PRESETS[command]
    elif command == "help":
        commands()
    elif command == "quit":
        exit()
    else:
        print("Invalid command")

# This function facilitates the user's selection of a preset molecular structure from the dictionary
# It first iterates through the presets dictionary and assigns each one a number
# Hence it provides the user with a list of numbered choices and prompts them to choose one
# Using a 'while true' loop to ensure that the program will not continue without a valid input
def select_preset():
    print("Available presets:")
    for index, preset_name in enumerate(PRESETS.keys(), start=1):
        print(f"{index}. {preset_name}")

    while True:
        try:
            choice = int(input("Enter the number corresponding to the preset: "))
            if 1 <= choice <= len(PRESETS):
                preset_name = list(PRESETS.keys())[choice - 1]
                return preset_name
            else:
                print("Please enter a valid number.")
        except ValueError:
            print("Please enter a number.")

# This function allows the user to input their desired number of carbons
def get_n(min, prompt="Please enter number of carbons: "):
    """
    Handle user input of an integer with a specified minimum value
    """
    valid = False
    while not valid:
        try:
            n = int(input(prompt)) # attempting to convert the user input to an integer
            if n < min:
                raise Exception() # raising an exception if the value is less than the specified minimum
            else:
                valid  = True
        except ValueError:
            print("Invalid input") # catching errors where the entered value cannot be converted to an integer
        except Exception:
            print("Value must be at least %s" % min) # exception handling for when the entered value is less than the specified minimum
    return n

# Main program loop
# First displays welcome message and available commands, prompting the user to input a command
# If 'linear' or 'cyclic' is chosen, the user is prompted to enter the number of carbons, n
# If 'presets' is chosen, the user is provided with a list of available presets and prompted to choose one
# If 'quit' is chosen, the program exits
# The program will then print the energy eigenvalues and associated degeneracies
# The 'while true' loop ensures that the program does not continue until the user inputs a valid command
def main():
    while True:
        welcome_message()
        commands()
        command = input("").lower().strip()

        if command == "linear" or command == "cyclic":
            n = get_n(2 if command == "linear" else 3)
            connectivities = connectivity_input(command, n)
            print_results(connectivities)

        elif command == "presets":
            preset = select_preset()
            connectivities = connectivity_input(preset, None)
            print_results(connectivities)

        elif command == "quit":
            exit()
        else:
            print("Invalid command")
            continue

        input("Press enter to continue...")

if __name__ == "__main__":
    main()

----------------
Program to determine the Huckel π-energies and degeneracies for a range of molecules
----------------
Available commands:

linear	to determine the Huckel π-energies and degeneracies for a linear polyene with n carbons
cyclic	to determine the Huckel π-energies and degeneracies for a cyclic polyene with n carbons
presets	to determine the Huckel π-energies and degeneracies for a set of preset molecules including certain sp2-hybridized Platonic solids and Buckminsterfullerene
help	to display this list of commands
quit	to exit the program
presets
Available presets:
1. tetrahedron
2. octahedron
3. cube
4. icosahedron
5. dodecahedron
6. fullerene60
Enter the number corresponding to the preset: 1
Energy     Degeneracy
α-1.000*β     3     
α+3.000*β     1     
Press enter to continue...
----------------
Program to determine the Huckel π-energies and degeneracies for a range of molecules
----------------
Available commands:

linear	to determine the Huckel π-energies and degenera